<a href="https://colab.research.google.com/github/cyrus2281/notes/blob/main/MachineLearning/Generative_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content

>[Content](#scrollTo=9CWyV4beAH0q)

>[Generative AI](#scrollTo=d95n8HFhAJ3m)

>>[OpenAI](#scrollTo=i4ToZgP9-rIo)

>>>[Few shot Prompt](#scrollTo=wW5Py1HkneoI)

>>>[Function Calling](#scrollTo=_sHjRxqRu1SE)



# Generative AI

## OpenAI

In [9]:
!pip install openai

In [14]:
import openai
print(openai.__version__)

1.14.1


In [6]:
openai_api_key = "" # @param {type:"string"}

In [16]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ], # Roles: system, user, assistant, tool
  max_tokens=128, # Maximum number of tokens you want result
  temperature=1, # How creative the output should be
  n=2 # Number of the outputs
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, a concept lives and breathes,\nWhere functions call themselves, like whispers in the trees.\nRecursion is the art, a trance-like dance,\nUnfolding mysteries with each recursive glance.\n\nLike a mirror reflecting its own reflection,\nA function loops back, with no objection.\nIt breaks down a problem, into smaller parts,\nAnd solves them one by one, with gentle smarts.\n\nA journey through layers, like a dream unfurled,\nEach invocation a thread in the code world.\nIt reaches the base case, like a destination,\nUnwinding the stack, with jubilation.\n\nSo embrace recursion, with courage and grace,\nLet it guide your code through time and space.\nFor in its elegant cycle, we find our place,\nA recursive enchantment, woven with grace.', role='assistant', function_call=None, tool_calls=None)


**This is an example of zero shot prompt**

### Few shot Prompt


In [5]:
person_info="My name is Cyrus, I'm a software engineer at X, and I live in Canada."

In [4]:
prompt = f'''
Extract the following information from the given text and return it as a JSON object:

name
job
company
country

This is the body of text to extract the information from:
{person_info}
'''

In [18]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message.content)

{
  "name": "Cyrus",
  "job": "software engineer",
  "company": "X",
  "country": "Canada"
}


**This is an example of few shot prompt**

### Function Calling

In [19]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


In [21]:
# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [25]:
# Step 2: check if the model wanted to call a function
if tool_calls:
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        print(tool_call)
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
    )  # get a new response from the model where it can see the function response
    print(second_response.choices[0].message.content)

ChatCompletionMessageToolCall(id='call_Q6tNs8Yd24ROxqmgrMYYYAC1', function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_efCZ1xEyMhMqKCVvFNsU3gax', function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_UgPOHn0Gls5GEtPiNdYlseoB', function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), type='function')
Currently in San Francisco, the weather is 72°C. In Tokyo, it is 10°C, and in Paris, it is 22°C.
